In [215]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np

In [216]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [217]:
df = pd.read_csv('../../../datos/july_23/2023-07-01.csv')

In [218]:
df

,block_timestamp,nonce,from_address,to_address,value,gas
0,2023-07-01 00:06:47+00:00,461958,0x292f04a44506c2fd49bac032e1ca148c35a478c8,0xf1a7b0f702eaf433e4697b766306bca41985b75a,881375944790000,55000
1,2023-07-01 00:06:47+00:00,129,0x5c80ae7a02b36c392e34f28859c9dc7a4ecb544f,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,6000000000000000,205047
2,2023-07-01 00:06:47+00:00,38,0xc845914efd144c75d2ffeec2f9609b3bebad681e,0x82e0b8cdd80af5930c4452c684e71c861148ec8a,6000000000000000,202640
3,2023-07-01 00:06:47+00:00,199,0x2bf23d3ab2fe916232ac836b42dc542884df45ae,0x881d40237659c251811cec9c364ef91dc08d300c,300000000000000000,352654
4,2023-07-01 00:33:47+00:00,30,0x78d0365355790d602b571dca709564239954aef6,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,10000000000000000,401234
...,...,...,...,...,...,...
514676,2023-07-01 00:58:11+00:00,0,0xc4336b03df3ecd8abcac566721156341409c33d9,0x32400084c286cf3e17e7b677ea9583e60a000324,8933120288488970,149210
514677,2023-07-01 01:19:35+00:00,0,0xe38af4ac404011282e662653286171f07a804bd2,0x32400084c286cf3e17e7b677ea9583e60a000324,146283814745135310,149210
514678,2023-07-01 01:46:23+00:00,1,0xb0394a77f62c2917a5c7546b9fee0bc57a80df71,0x32400084c286cf3e17e7b677ea9583e60a000324,31884667511922704,149210
514679,2023-07-01 04:27:35+00:00,0,0xd6d60b9e1579db6ecfdabb58715bb5eba7705e2d,0x32400084c286cf3e17e7b677ea9583e60a000324,8812306457800690,149210


In [219]:
df.columns

Index(['block_timestamp', 'nonce', 'from_address', 'to_address', 'value',
       'gas'],
      dtype='object')

In [220]:
len(set(df['from_address']).union(set(df['to_address'])))

310627

In [221]:
df = df[~df.to_address.isna()]

In [222]:
ids = {}

for i, id in enumerate(set(df['from_address']).union(set(df['to_address']))):
    ids[id] = i

In [223]:
ids

{'0x49ce124382a4df24497fffef25e20fe41d45b188': 0,
 '0xfb7cbb692e335978bf1596b8bb9ca810840578d4': 1,
 '0x05269d61eb7b97a6473d3754090e9afb3b9bf92d': 2,
 '0x1d57e57a07b092eb4351611b3f2522bf1555eb77': 3,
 '0xf95b8114495026193b91e17e0af6e11c4aa49125': 4,
 '0xb33259935b898319829c9e958dfe0af78738728f': 5,
 '0x1e8559399379be4385d6f3f2145cf81319f248c5': 6,
 '0xc99947e2eb20d9d4d7655695e656e9ab19b70f86': 7,
 '0xfc16c134aaaf6beec98619f9743ea318ea7aea78': 8,
 '0xbd101fffac618cc704f005315143dd63b445c5e7': 9,
 '0x9544beee7f06b36a813aaa352e46a5702ea3c64e': 10,
 '0x410c70170b1f5f091406b130cc71763c9638c7d0': 11,
 '0x65fd9fca0c00c0478f08b30b788de215b1b541ff': 12,
 '0xc2bae7b3286b55d22ab3d82255359ae378c530da': 13,
 '0x42ddbbf013aa0ae8fa0ea82613e303cbaf9c5596': 14,
 '0x33217c73c5de2102069e647faa0cf68642031483': 15,
 '0xb650fac5e89d3a0f0c1b5c92cbd58d3f1250afda': 16,
 '0x87d048796104f2aac317fa741bfe18db7f6084ad': 17,
 '0x25334db72b8b093566eb062c508cf901f9c0ffc8': 18,
 '0xd73b09be932729ad57700147fe6100ecebbbe

In [224]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, BatchNormalization, Lambda

# el 1 representa a cualquier indice, un tensor de una dimensión
input_aux = Input(1)

# el 128 es arbitrario, podría ser de cualquier valor
x = Embedding(len(ids), 128)(input_aux)

# TODO: investigar que es esto
x = Dense(64, activation='tanh')(x)

# lleva todos los valores entre -1 y 1
output_aux = BatchNormalization()(x)

model_aux = Model(input_aux, output_aux)
model_aux.summary()

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_57 (InputLayer)       [(None, 1)]               0         
                                                                 
 embedding_14 (Embedding)    (None, 1, 128)            39759744  
                                                                 
 dense_14 (Dense)            (None, 1, 64)             8256      
                                                                 
 batch_normalization_14 (Ba  (None, 1, 64)             256       
 tchNormalization)                                               
                                                                 
Total params: 39768256 (151.70 MB)
Trainable params: 39768128 (151.70 MB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [225]:

# son los 3 inputs que representan las 3 cabezas de la red, lo que se conoce como siamesa
input_layer_anchor = Input(1)
input_layer_positive = Input(1)
input_layer_negative = Input(1)

x_a = model_aux(input_layer_anchor)
x_p = model_aux(input_layer_positive)
x_n = model_aux(input_layer_negative)

# un tensor es ... investigar bien. Puede ser cualquier cosa, literalmente.
merged_output = Lambda(lambda tensors: tf.stack(tensors, axis=-1))([x_a, x_p, x_n])

model = Model([input_layer_anchor, input_layer_positive, input_layer_negative], merged_output)

In [226]:
class TripletCustom(tf.keras.losses.Loss):
    def __init__(self, margin=1.0, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    @tf.function
    def __call__(self, y, y_pred, sample_weight=None):
        anchor, positive, negative =  tf.split(y_pred, num_or_size_splits=3, axis=-1)
        
        distance_positive = tf.norm(anchor - positive, axis=-1)
        distance_negative = tf.norm(anchor - negative, axis=-1)

        loss = tf.maximum(distance_positive - distance_negative + self.margin, 0.0)
        resultado = tf.reduce_mean(loss)

        sess = tf.Session(distance_positive)

        with sess.as_default():
            if tf.math.is_nan(loss) or tf.math.is_nan(resultado):
                print("Entro aca")
                tf.print(distance_negative)
                print(distance_positive)
                print(loss)
                print(resultado)
        return resultado

loss = TripletCustom()    

model.compile(
    optimizer=Adam(0.001),
    loss=loss
)

In [227]:
from keras.utils import Sequence
import numpy as np

class GeneratorTriplet(Sequence):
    def __init__(self, df, ids, batch_size):
        self.df = df
        self.act_index = 0
        self.ids = ids
        self.batch_size = batch_size
        self.limit = int(np.ceil(len(self.df) / self.batch_size))

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.act_index < self.limit:
            resultado = self.__getitem__(self.act_index)
            self.act_index += 1
            return resultado
        else:
            raise StopIteration

    def __getitem__(self, index):
        # el Anchor es un A, el positive es un B y el negative es un C. Queremos acercar A y B tanto como sea posible alejando a C. Podría suceder que sea un vecino, pero la probabilidad es baja porque es al azar.
        # Si llegara a elegir algún vecino mal, de casualidad, en una siguiente epoch debería corregirse o inclusive en una misma epoch en una siguiente iteración del generador.
        try:
            init = index * self.batch_size
            end = (index + 1) * self.batch_size

            batch = self.df[init:end]
            negative = self.df.sample(len(batch))

            anchor = np.array(batch['from_address'].apply(lambda x: self.ids.get(x)))
            positive = np.array(batch['to_address'].apply(lambda x: self.ids.get(x)))
            negative = np.array(negative['to_address'].apply(lambda x: self.ids.get(x)))
            
            anchor = tf.convert_to_tensor(anchor)
            positive = tf.convert_to_tensor(positive)
            negative = tf.convert_to_tensor(negative)
            
            # el fake target simunla lo que sería aprendizaje supervisado        
            fake_target = tf.convert_to_tensor(np.array([1]*self.batch_size))

            return ([anchor, positive, negative], [fake_target]*3)
        except e:
            print("Esta fallando aca")
            print(e)

In [228]:
generator = GeneratorTriplet(df, ids, 64)

In [229]:
model.fit(generator, epochs=2)

Epoch 1/2


AttributeError: in user code:

    File "/home/nahuel/.cache/pypoetry/virtualenvs/pow_a_pos-K56kznNs-py3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_31383/3288962294.py", line 16, in __call__  *
        sess = tf.Session()

    AttributeError: module 'tensorflow' has no attribute 'Session'


In [ ]:
# input_layer = Input(shape=(2,))
# dense_layer = Dense(units=1, activation='linear')(input_layer)
# model = Model(inputs=input_layer, outputs=dense_layer)
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.summary()
# y_train = np.random.rand(100)
# print(model.fit(combined_tensor, y_train, epochs=1, batch_size=32))